# Why is Julia fast?

Or rather, how *can* Julia be fast -- since it's perfectly possible to write slow code in Julia.

The point is that if you start with a slow piece of Julia code, it is very possible that by following a few rules, you can tweak it a bit and rapidly end up with a piece of Julia code that is almost identical, but now blindingly fast -- roughly as fast as optimized C! 

How can this be possible? A large part of the reason is through the clever design of the language, and how it handles **types**.

Although it is possible to get a long way in Julia without worrying about, thinking about, or even mentioning types, much of the power of Julia resides in the way it deals with types

## What is a type?

A type is a label that tells the computer how to interpret a block of data in memory. Julia allows us to play around with that to some extent:

In [2]:
n = Int('A')

65

In [4]:
typeof(n)

Int64

In [3]:
bitstring(n)

"0000000000000000000000000000000000000000000000000000000001000001"

In [5]:
reinterpret(Float64, n)

3.2e-322

In [7]:
reinterpret(UInt8, [n])

8-element reinterpret(UInt8, ::Array{Int64,1}):
 0x41
 0x00
 0x00
 0x00
 0x00
 0x00
 0x00
 0x00

In [1]:
x = 0.1

0.1

In [2]:
typeof(x)

Float64

In [3]:
bits(x)

"0011111110111001100110011001100110011001100110011001100110011010"

In [4]:
y = reinterpret(Int64, x)

4591870180066957722

In [ ]:
bits(y)

The exact same bit pattern is interpreted in two completely different ways. Indeed, we can even interpret this as a [fixed size] vector of two `Int32`:

In [5]:
using StaticArrays
reinterpret(SVector{2, Int32}, [x])

1-element Array{StaticArrays.SArray{Tuple{2},Int32,1,2},1}:
 Int32[-1717986918, 1069128089]

## Code specialization

Suppose we define a function in Julia and apply it to values of different types:

In [10]:
3

3

In [12]:
3 + 4    # add_int(3, 4)

7

In [11]:
@which 3 + 4

+(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:53

In [13]:
3.1 + 4.2

7.300000000000001

In [14]:
@which 3.1 + 4.2   # add_float(3.1, 4.2)

+(x::Float64, y::Float64) in Base at float.jl:395

"Despacho múltiple" -- multiple dispatch

Escoge una versión de la función ("dispatch") según los tipos de todos los argumentos que le estoy pasando ("multiple")


In [9]:
typeof(3.0)

Float64

In [ ]:
3.0 + 4.0

In [15]:
f(x, y) = x * y 

f (generic function with 1 method)

In [16]:
f

f (generic function with 1 method)

In [17]:
+

+ (generic function with 163 methods)

In [18]:
methods(+)

# 163 methods for generic function "+":
[1] +(x::Bool, z::Complex{Bool}) in Base at complex.jl:277
[2] +(x::Bool, y::Bool) in Base at bool.jl:104
[3] +(x::Bool) in Base at bool.jl:101
[4] +(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:112
[5] +(x::Bool, z::Complex) in Base at complex.jl:284
[6] +(a::Float16, b::Float16) in Base at float.jl:392
[7] +(x::Float32, y::Float32) in Base at float.jl:394
[8] +(x::Float64, y::Float64) in Base at float.jl:395
[9] +(z::Complex{Bool}, x::Bool) in Base at complex.jl:278
[10] +(z::Complex{Bool}, x::Real) in Base at complex.jl:292
[11] +(::Missing, ::Missing) in Base at missing.jl:96
[12] +(::Missing) in Base at missing.jl:83
[13] +(::Missing, ::Number) in Base at missing.jl:97
[14] +(level::Base.CoreLogging.LogLevel, inc::Integer) in Base.CoreLogging at logging.jl:106
[15] +(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:406
[16] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:434
[17] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:433
[18] +(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:432
[19] +(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:403
[20] +(x::BigInt, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.GMP at gmp.jl:440
[21] +(x::BigInt, c::Union{Int16, Int32, Int64, Int8}) in Base.GMP at gmp.jl:446
[22] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:556
[23] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:549
[24] +(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:543
[25] +(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:402
[26] +(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:371
[27] +(x::BigFloat, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.MPFR at mpfr.jl:378
[28] +(x::BigFloat, c::Union{Int16, Int32, Int64, Int8}) in Base.MPFR at mpfr.jl:386
[29] +(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:394
[30] +(x::Dates.CompoundPeriod, y::Dates.CompoundPeriod) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/periods.jl:343
[31] +(x::Dates.CompoundPeriod, y::Dates.Period) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/periods.jl:341
[32] +(x::Dates.CompoundPeriod, y::Dates.TimeType) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/periods.jl:371
[33] +(x::Dates.Date, y::Dates.Day) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:74
[34] +(x::Dates.Date, y::Dates.Week) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:72
[35] +(dt::Dates.Date, z::Dates.Month) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:55
[36] +(dt::Dates.Date, y::Dates.Year) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:28
[37] +(dt::Dates.Date, t::Dates.Time) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:19
[38] +(t::Dates.Time, dt::Dates.Date) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:20
[39] +(x::Dates.Time, y::Dates.TimePeriod) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:78
[40] +(dt::Dates.DateTime, z::Dates.Month) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:48
[41] +(dt::Dates.DateTime, y::Dates.Year) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:24
[42] +(x::Dates.DateTime, y::Dates.Period) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:76
[43]

In [19]:
@which (1 + im) + (3 + im)

+(z::Complex, w::Complex) in Base at complex.jl:266

In [20]:
f(x, y) = x * y

f (generic function with 1 method)

In [21]:
f(3, 4)    

12

In [22]:
typeof(ans)

Int64

In [23]:
f(3.5, 4.5)     

15.75

In [24]:
typeof(ans)

Float64

As we saw above, the internal representations of `Int64`s and `Float64`s are very different, so Julia must be calling different functions for the multiplication and addition.

Perhaps every time `f` is called, it is doing the analysis (at "run-time"); this is what Python does, for example.

In fact, though, Julia is **much more clever** than this: when you call `f` with a new combination of types, **Julia compiles a new specialized version of `f` for that "type signature"**. The next time that `f` is called with that specific combination of types, Julia just reuses the already-compiled specialized version.

Julia performs **type inference** on the code, in which the input types are propagated through the code, so that Julia can work out what the type of each variable in the function is.

Furthermore, since everything is ["just-in-time" or "ahead-of-time" compiled](https://www.youtube.com/watch?v=7KGZ_9D_DbI), this all happens, in the best case, **at compile time**. 

### Introspection

We can make Julia tell us some of this via its excellent **introspection** capabilities:

In [26]:
g(x, y) = x + x * y

g (generic function with 1 method)

In [27]:
@code_lowered g(3, 4)

CodeInfo(
1 ─ %1 = x * y
│   %2 = x + %1
└──      return %2
)

In [29]:
@code_typed g(3, 4)

CodeInfo(
1 ─ %1 = (Base.mul_int)(x, y)::Int64
│   %2 = (Base.add_int)(x, %1)::Int64
└──      return %2
) => Int64

In [31]:
@code_llvm g(3, 4)


;  @ In[26]:1 within `g'
define i64 @julia_g_13040(i64, i64) {
top:
; ┌ @ int.jl:54 within `*'
   %2 = mul i64 %1, %0
; └
; ┌ @ int.jl:53 within `+'
   %3 = add i64 %2, %0
; └
  ret i64 %3
}


In [32]:
@code_native g(3, 4)

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ In[26]:1 within `g'
; │┌ @ In[26]:1 within `*'
	decl	%eax
	imull	%edi, %esi
; │└
; │┌ @ int.jl:53 within `+'
	decl	%eax
	leal	(%esi,%edi), %eax
; │└
	retl
	nopl	(%eax)
; └


In [33]:
@code_native g(3.5, 4.5)

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ In[26]:1 within `g'
; │┌ @ In[26]:1 within `*'
	vmulsd	%xmm1, %xmm0, %xmm1
; │└
; │┌ @ float.jl:395 within `+'
	vaddsd	%xmm0, %xmm1, %xmm0
; │└
	retl
	nopl	(%eax)
; └


We see that Julia has been able to infer what the types of each variable are, and what type is returned from this version of the function.

In [14]:
@code_native f(3.5, 4.5)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[6]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 1
	mulsd	%xmm1, %xmm0
	popq	%rbp
	retq
	nopw	(%rax,%rax)


Note that `f(3, 4.5)` is a different case, where we are mixing two types.

In [15]:
@code_native f(3, 4.5)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[6]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 1
	xorps	%xmm1, %xmm1
	cvtsi2sdq	%rdi, %xmm1
	mulsd	%xmm1, %xmm0
	popq	%rbp
	retq
	nopw	%cs:(%rax,%rax)


In [34]:
using BenchmarkTools

In [35]:
@btime g(3, 4)

  0.017 ns (0 allocations: 0 bytes)


15

In [38]:
function g(x, y)
    if (rand() < 0.5)
        x = Float64(x)
    end
    return x + x * y
end

g (generic function with 1 method)

In [40]:
@code_warntype g(3, 4)

Body::Union{Float64, Int64}
1 ── %1  = Random.GLOBAL_RNG::Random.MersenneTwister
│    %2  = (Base.getfield)(%1, :idxF)::Int64
│    %3  = Random.MT_CACHE_F::Int64
│    %4  = (%2 === %3)::Bool
└───       goto #3 if not %4
2 ── %6  = $(Expr(:gc_preserve_begin, :(%1)))
│    %7  = (Base.getfield)(%1, :state)::Random.DSFMT.DSFMT_state
│    %8  = (Base.getfield)(%1, :vals)::Array{Float64,1}
│    %9  = $(Expr(:foreigncall, :(:jl_array_ptr), Ptr{Float64}, svec(Any), :(:ccall), 1, :(%8)))::Ptr{Float64}
│    %10 = (Base.getfield)(%1, :vals)::Array{Float64,1}
│    %11 = (Base.arraylen)(%10)::Int64
│          invoke Random.dsfmt_fill_array_close1_open2!(%7::Random.DSFMT.DSFMT_state, %9::Ptr{Float64}, %11::Int64)
│          $(Expr(:gc_preserve_end, :(%6)))
└───       (Base.setfield!)(%1, :idxF, 0)
3 ┄─       goto #4
4 ── %16 = (Base.getfield)(%1, :vals)::Array{Float64,1}
│    %17 = (Base.getfield)(%1, :idxF)::Int64
│    %18 = (Base.add_int)(%17, 1)::Int64
│          (Base.setfield!)(%1, :idxF, %18)


In [41]:
@btime g(3, 4)

  10.393 ns (0 allocations: 0 bytes)


15.0

In [16]:
g(x) = 3x

g (generic function with 1 method)

In [17]:
v = [1, 2, 3]

3-element Array{Int64,1}:
 1
 2
 3

In [18]:
w = [1, 3, 4.5]

3-element Array{Float64,1}:
 1.0
 3.0
 4.5

In [19]:
[1, "hello"]

2-element Array{Any,1}:
 1       
  "hello"

In [21]:
using DataFrames

In [22]:
[1, missing]

2-element Array{Union{Int64, Missings.Missing},1}:
 1       
  missing

In [ ]:
g.(v)

In [23]:
promote(1, 3.5)

(1.0, 3.5)

In [24]:
promote_type(Int64, Float64)

Float64

In [25]:
promote(1, missing)

(1, missing)

In [26]:
typeof(ans)

Tuple{Int64,Missings.Missing}

In [27]:
supertype(Int64)

Signed

In [28]:
supertype(Signed)

Integer

In [29]:
supertype(Integer)

Real

In [31]:
supertype(Real)

Number

In [32]:
supertype(Number)

Any

In [33]:
1 isa Integer

true

In [34]:
1 isa Real

true

In [35]:
1 isa Complex

false

In [37]:
1 isa Any

true

In [38]:
missing isa Any

true

In [39]:
[1, 3.5+4im]

2-element Array{Complex{Float64},1}:
 1.0+0.0im
 3.5+4.0im

In [40]:
v = [1, missing]

2-element Array{Union{Int64, Missings.Missing},1}:
 1       
  missing

In [41]:
push!(v, "hello")

LoadError: [91mMethodError: Cannot `convert` an object of type String to an object of type Int64
This may have arisen from a call to the constructor Int64(...),
since type constructors fall back to convert methods.[39m

# Type stability

For the above to work, Julia must be able to successfully complete type inference and find a unique type for each variable in the function, otherwise there is a serious performance penalty. [This is being mitigated in Julia 0.7.]

For example, here is a "classic" example where type inference can fail:

In [42]:
function sum1(n)
    s = 0
    
    for i in 1:n
        s += i / 2
    end
    
    return s
end

sum1 (generic function with 1 method)

If we just run the function, everything seems to go smoothly:

In [43]:
sum1(10)

27.5

But now we time it:

In [44]:
@time sum1(10)

  0.000005 seconds (113 allocations: 6.561 KiB)


27.5

In [46]:
@time sum1(10)

  0.000004 seconds (34 allocations: 640 bytes)


27.5

In [48]:
using BenchmarkTools

In [50]:
b = @benchmark sum1(10)

BenchmarkTools.Trial: 
  memory estimate:  464 bytes
  allocs estimate:  29
  --------------
  minimum time:     208.178 ns (0.00% GC)
  median time:      234.747 ns (0.00% GC)
  mean time:        267.680 ns (7.14% GC)
  maximum time:     2.990 μs (80.78% GC)
  --------------
  samples:          10000
  evals/sample:     555

In [51]:
b.times

10000-element Array{Float64,1}:
  208.178
  208.222
  208.344
  208.36 
  208.463
  208.467
  208.47 
  208.474
  208.577
  208.587
  208.591
  208.595
  208.6  
    ⋮    
 2541.49 
 2560.68 
 2577.94 
 2629.27 
 2631.94 
 2735.47 
 2752.34 
 2759.8  
 2760.45 
 2837.11 
 2984.13 
 2989.81 

In [52]:
using Plots; gr()

Plots.GRBackend()

In [53]:
histogram(b.times, bins=200)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 500 
 
 
 1000 
 
 
 1500 
 
 
 2000 
 
 
 2500 
 
 
 3000 
 
 
 0 
 
 
 1000 
 
 
 2000 
 
 
 3000 
 
 
 4000 
 
 
 5000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

There is no reason for this simple function to be allocating memory. What is going on? Let's use introspection:

In [54]:
@code_warntype sum1(10)

Variables:
  #self# <optimized out>
  n::Int64
  i::Int64
  #temp#@_4::Int64
  s::Union{Float64, Int64}
  #temp#@_6::Core.MethodInstance
  #temp#@_7::Float64

Body:
  begin 
      s::Union{Float64, Int64} = 0 # line 4:
      SSAValue(2) = (Base.select_value)((Base.sle_int)(1, n::Int64)::Bool, n::Int64, (Base.sub_int)(1, 1)::Int64)::Int64
      #temp#@_4::Int64 = 1
      5: 
      unless (Base.not_int)((#temp#@_4::Int64 === (Base.add_int)(SSAValue(2), 1)::Int64)::Bool)::Bool goto 30
      SSAValue(3) = #temp#@_4::Int64
      SSAValue(4) = (Base.add_int)(#temp#@_4::Int64, 1)::Int64
      i::Int64 = SSAValue(3)
      #temp#@_4::Int64 = SSAValue(4) # line 5:
      unless (s::Union{Float64, Int64} isa Int64)::Bool goto 15
      #temp#@_6::Core.MethodInstance = MethodInstance for +(::Int64, ::Float64)
      goto 24
      15: 
      unless (s::Union{Float64, Int64} isa Float64)::Bool goto 19
      #temp#@_6::Core.MethodInstance = MethodInstance for +(::Float64, ::Float64)
      goto 24
      

The output of `@code_warntype` is not always very readable, but anything in <font color="red"> red </font> is bad news: it signals a **type instability**, where Julia is unable to determine a unique type for a variable.

Very recently, a new package has come out that gives us a more user-friendly version of this information:

In [55]:
using Traceur

In [56]:
@trace sum1(10)

(sum1)(::Int64) at In[42]:2
  s is assigned as Int64 at line 2
  s is assigned as Float64 at line 5
  dynamic dispatch to s + (Base.div_float)((Base.sitofp)(Float64, i), (Base.sitofp)(Float64, 2)) at line 5
  returns Union{Float64, Int64}


27.5

We see that the variable `s` is the culprit: it is first an `Int64` and then a `Float64`. Julia is thus unable to determine which type it should actually be, so that the variable will be **boxed** and leads to **dynamic dispatch**, in which the exact version of the function is determined at run-time. This usually represents the death of good performance (cf. Python, which does exactly this).

In this case, the solution is simple: we initialize `s` as a `Float64` from the start:

In [57]:
function sum2(n)
    s = 0.0  
    
    for i in 1:n
        s += i / 2
    end 
    
    return s
end

sum2 (generic function with 1 method)

In [60]:
sum1(1)
@time sum1(10^6)

  0.030245 seconds (3.00 M allocations: 45.777 MiB, 27.73% gc time)


2.5000025e11

In [61]:
sum2(1) 
@time sum2(10^6)   

  0.000984 seconds (6 allocations: 192 bytes)


2.5000025e11

**Exercise**: To what extent has this effect been mitigated in Julia 0.7? (Note that currently only the REPL works with Julia 0.7; neither IJulia nor Juno work stably yet.)

## Global variables: just don't

This is the reason why global variables are bad news in Julia: their type could change, in principle, at any time, and so they *always* lead to type instability.

Except, that is, if they are declared as `const`. This, confusingly, does not mean that their *values* can't change, but rather that their types can't change.

### Don't use globals; declare constants as `const`

In [62]:
a = 1
f() = a
 
@code_warntype f() 

Variables:
  #self# <optimized out>

Body:
  begin 
      return Main.a
  end::Any


In [63]:
const cc = 1
g() = cc

@code_warntype g()

Variables:
  #self# <optimized out>

Body:
  begin 
      return Main.cc
  end::Int64


## Summary

- We **can** talk about types (and should bear them in mind)


- But don't **have to** talk explicitly about types



- Julia **infers** types

In [ ]:
function sum1_new(N::Int64)
    ...
end

In [ ]:
function sum1_new(N::Float64)
    ...
end

In [64]:
+

+ (generic function with 201 methods)

In [65]:
methods(+)

# 201 methods for generic function "+":
+(x::Bool, z::Complex{Bool}) in Base at complex.jl:232
+(x::Bool, y::Bool) in Base at bool.jl:89
+(x::Bool) in Base at bool.jl:86
+(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:96
+(x::Bool, z::Complex) in Base at complex.jl:239
+(a::Float16, b::Float16) in Base at float.jl:372
+(x::Float32, y::Float32) in Base at float.jl:374
+(x::Float64, y::Float64) in Base at float.jl:375
+(z::Complex{Bool}, x::Bool) in Base at complex.jl:233
+(z::Complex{Bool}, x::Real) in Base at complex.jl:247
+(x::Char, y::Integer) in Base at char.jl:40
+(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:312
+(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:334
+(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:327
+(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:321
+(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:289
+(x::BigInt, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.GMP at gmp.jl:346
+(x::BigInt, c::Union{Int16, Int32, Int64, Int8}) in Base.GMP at gmp.jl:362
+(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:460
+(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:453
+(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:447
+(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:308
+(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:277
+(x::BigFloat, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.MPFR at mpfr.jl:284
+(x::BigFloat, c::Union{Int16, Int32, Int64, Int8}) in Base.MPFR at mpfr.jl:292
+(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:300
+(B::BitArray{2}, J::UniformScaling) in Base.LinAlg at linalg/uniformscaling.jl:59
+(a::Base.Pkg.Resolve.VersionWeights.VWPreBuildItem, b::Base.Pkg.Resolve.VersionWeights.VWPreBuildItem) in Base.Pkg.Resolve.VersionWeights at pkg/resolve/versionweight.jl:87
+(a::Base.Pkg.Resolve.VersionWeights.VWPreBuild, b::Base.Pkg.Resolve.VersionWeights.VWPreBuild) in Base.Pkg.Resolve.VersionWeights at pkg/resolve/versionweight.jl:135
+(a::Base.Pkg.Resolve.VersionWeights.VersionWeight, b::Base.Pkg.Resolve.VersionWeights.VersionWeight) in Base.Pkg.Resolve.VersionWeights at pkg/resolve/versionweight.jl:197
+(a::Base.Pkg.Resolve.MaxSum.FieldValues.FieldValue, b::Base.Pkg.Resolve.MaxSum.FieldValues.FieldValue) in Base.Pkg.Resolve.MaxSum.FieldValues at pkg/resolve/fieldvalue.jl:44
+(x::Base.Dates.CompoundPeriod, y::Base.Dates.CompoundPeriod) in Base.Dates at dates/periods.jl:349
+(x::Base.Dates.CompoundPeriod, y::Base.Dates.Period) in Base.Dates at dates/periods.jl:347
+(x::Base.Dates.CompoundPeriod, y::Base.Dates.TimeType) in Base.Dates at dates/periods.jl:387
+(x::Date, y::Base.Dates.Day) in Base.Dates at dates/arithmetic.jl:77
+(x::Date, y::Base.Dates.Week) in Base.Dates at dates/arithmetic.jl:75
+(dt::Date, z::Base.Dates.Month) in Base.Dates at dates/arithmetic.jl:58
+(dt::Date, y::Base.Dates.Year) in Base.Dates at dates/arithmetic.jl:32
+(dt::Date, t::Base.Dates.Time) in Base.Dates at dates/arithmetic.jl:20
+(t::Base.Dates.Time, dt::Date) in Base.Dates at dates/arithmetic.jl:24
+(x::Base.Dates.Time, y::Base.Dates.TimePeriod) in Base.Dates at dates/arithmetic.jl:81
+(dt::DateTime, z::Base.Dates.Month) in Base.Dates at dates/arithmetic.jl:52
+(dt::DateTime, y::Base.Dates.Year) in Base.Dates at dates/arithmetic.jl:28
+(x::DateTime, y::Base.Dates.Period) in Base.Dates at dates/arithmetic.jl:79
+(d::Missings.Missing) in Missings at /Users/dpsanders/.julia/v0.6/Missings/src/Missings.jl:92
+(::Missings.Missing, ::Missings.Missing) in Missings at /Users/dpsanders/.julia/v0.6/Missings/src/Missings.jl:107
+(d::Missings.Missing, x::Number) in Missings at /Users/dpsanders/.julia/v0.6/Missings/src/Missings.jl:108
+(x::ASTInterpreter2.JuliaProgramCounter, y::Integer) in ASTInterpreter2 at /Users/dpsanders/.julia/v0.6/ASTInterpreter2/src/ASTInterpreter2.jl:18
+(y::AbstractFloat, x::Bool) in Base at bool.jl:98
+(x::T, y::T) whe

In [42]:
]add Traceur

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %       ]  82.2 % Resolving package versions...
  Updating `~/.julia/environments/v1.1/Project.toml`
  [37b6cedf] + Traceur v0.3.0
  Updating `~/.julia/environments/v1.1/Manifest.toml`
 [no changes]


Para verificar "type stability" (estabilidad de tipos)

In [43]:
using Traceur

In [44]:
@trace g(3, 4)

┌ Warning: dynamic dispatch to ((Core.apply_type)(Base.Val, x))()
└ @ essentials.jl:728
┌ Warning: x is assigned as Int64
└ @ In[38]:2
┌ Warning: x is assigned as Float64
└ @ In[38]:3
┌ Warning: g returns Union{Float64, Int64}
└ @ In[38]:2


15.0

El truco para tener buen rendimiento es tener type stability: si mando objetos del mismo tipo, debe regresar objetos del mismo tipo